<a href="https://colab.research.google.com/github/linjiw/linjiw.github.io/blob/main/texture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/poincloud/

/content/drive/MyDrive/poincloud


In [ ]:
%ls

In [4]:
LIB_PTH = "dtd/"
img = "images"
labels = "labels"

In [5]:
import os
import pandas as pd
from torchvision.io import read_image
import torch
from torch.utils.data import Dataset
from torchvision import datasets
# from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import torchvision
import torchvision.models as models
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torch.optim as optim
from sklearn.metrics import accuracy_score
from datetime import datetime
import numpy as np

# from torchvision import transforms

In [6]:
def getmp(LIB_PTH,img):
  kind = os.listdir(os.path.join(LIB_PTH,img))
  mp = {}
  count = 0
  for i in kind:
    
    mp[i] = count
    count+=1
  # print(count)
  return mp
mp = getmp(LIB_PTH,img)

In [7]:
def getcsv(LIB_PTH,img,mp):
  filename = "train_file.csv"
  test_name = "test_file.csv"
  file1 = open(filename,"w")
  file1.write(f"label,pth\n")
  file2 = open(test_name,"w")
  file2.write(f"label,pth\n")
  test_num = 12
  for i in os.listdir(os.path.join(LIB_PTH,img)):
    label = mp[i]
    count =0
    # test_num = 12

    for idx, j in enumerate(os.listdir(os.path.join(LIB_PTH,img,i))):
      if (j.endswith('.jpg') or j.endswith('.png') or j.endswith('.jpeg')) and idx>=test_num:
        file1.write(f"{label},{os.path.join(img,i,j)}\n")
        count+=1
      elif (j.endswith('.jpg') or j.endswith('.png') or j.endswith('.jpeg')):
        file2.write(f"{label},{os.path.join(img,i,j)}\n")
    # print(count)
  file1.close()
  file2.close()
  return filename
annotation = getcsv(LIB_PTH,img,mp)



In [9]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir=None, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 1])
        # img_path = self.img_labels.iloc[idx, 1]
        image = read_image(img_path)
        # print(image.shape)
        label = self.img_labels.iloc[idx, 0]
        if self.transform:
            image = self.transform(image)
            # print()
        if self.target_transform:
            label = self.target
        label = int(label)
        # print(type(label))
        return image,label

In [10]:
def compute_kl_loss(p, q, pad_mask=None):
    
    p_loss = torch.nn.functional.kl_div(torch.nn.functional.log_softmax(p, dim=-1), torch.nn.functional.softmax(q, dim=-1), reduction='none')
    q_loss = torch.nn.functional.kl_div(torch.nn.functional.log_softmax(q, dim=-1), torch.nn.functional.softmax(p, dim=-1), reduction='none')
    # p_loss = torch.nn.functional.kl_div(p,q, reduction='none')
    # q_loss = torch.nn.functional.kl_div(q,p, reduction='none')
 
    # pad_mask is for seq-level tasks
    if pad_mask is not None:
        p_loss.masked_fill_(pad_mask, 0.)
        q_loss.masked_fill_(pad_mask, 0.)

    # You can choose whether to use function "sum" and "mean" depending on your task
    p_loss = p_loss.mean()
    q_loss = q_loss.mean()

    loss = (p_loss + q_loss) / 2
    return loss
def train(args, model, device, train_samples, optimizer,scheduler, criterion, epoch):
    model.train()
    optimizer.zero_grad()
    train_loader = torch.utils.data.DataLoader(train_samples, batch_size=args['batch_size'], shuffle=True)
        # print(f"X.shape: {X.shape}")
    for batch_idx, (data, target) in enumerate(train_loader):
            # print(f"batch_idx: {batch_idx} data {data.shape}")
        data = data.to(device)
        # print(target)
        target = target.to(device)

        optimizer.zero_grad()
            
        output = model(data)
        # logits = model(data)
            
        # logits2 = model(data)

        #     # cross entropy loss for classifier
        # ce_loss = 0.5 * (criterion(logits, target) + criterion(logits2, target))

        # kl_loss = compute_kl_loss(logits, logits2)

        #     # carefully choose hyper-parameters
        # loss = ce_loss + 5 * kl_loss

        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args['log_interval'] == 0:
              print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))
    # scheduler.step()
    return loss.cpu().detach().numpy()
def test(args, model, device, dev_samples,criterion):
    model.eval()
    true_y_list = []
    pred_y_list = []
    with torch.no_grad():
        # for i in range(len(dev_samples)):
            # X, Y = dev_samples[i]

            # test_items = LibriItems(X, Y, context=args['context'])
        test_loader = torch.utils.data.DataLoader(dev_samples, batch_size=args['batch_size'], shuffle=False)

        for data, true_y in test_loader:
            data = data.to(device)
            true_y = true_y.to(device)                
                
            output = model(data)
            pred_y = torch.argmax(output, axis=1)
                
            loss = criterion(output, true_y)

            pred_y_list.extend(pred_y.tolist())
            true_y_list.extend(true_y.tolist())
    train_accuracy =  accuracy_score(true_y_list, pred_y_list)
    
    return train_accuracy , loss.cpu().detach().numpy()
def savemodel(model,epoch):
    now = datetime.now()
    name = now.strftime("%d_%m_%Y_%H_%M_%S")
    torch.save(model.state_dict(), f"model_epoch_{epoch}_{name}.pth")
def main(args):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)
    # model = Network(args['context']).to(device)
    # optimizer = optim.Adam(model.parameters(), lr=args['lr'])
    # model = args['model']
    train_transform = torchvision.transforms.Compose([
    torchvision.transforms.ToPILImage(),
    torchvision.transforms.ColorJitter(brightness=.5, hue=.3),
    torchvision.transforms.RandomPerspective(distortion_scale=0.6, p=1.0),
    torchvision.transforms.RandomRotation(degrees=(0, 180)),
    torchvision.transforms.RandomAffine(degrees=(30, 70), translate=(0.1, 0.3), scale=(0.5, 0.75)),
    torchvision.transforms.RandomAdjustSharpness(sharpness_factor=2),
    torchvision.transforms.RandomAutocontrast(),
    torchvision.transforms.RandomCrop((224,224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    
    ])
    test_transform = torchvision.transforms.Compose([
    torchvision.transforms.ToPILImage(),
    torchvision.transforms.RandomCrop((224,224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    
    ])
    num_classes = 47
    model = models.vgg16(pretrained=False)
    # model.fc = nn.Linear(512 , 47)
    model.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(p=0.2),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(p=0.2),
            nn.Linear(4096, num_classes),
        )
    model = model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=args['lr'], momentum=0.9)
    
    scheduler1 = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
    train_set = CustomImageDataset("train_file.csv", img_dir="dtd",transform=train_transform)
    test_set = CustomImageDataset("test_file.csv", img_dir="dtd",transform=test_transform)
    # train_set,test_set, val_set = torch.utils.data.random_split(dataset = allset,lengths = [allset.__len__()-200,100,100])


    criterion = torch.nn.CrossEntropyLoss()
    # If you want to use full Dataset, please pass None to csvpath
    # train_samples = LibriSamples(data_path = args['LIBRI_PATH'], shuffle=True, partition="train-clean-100", csvpath="./train_filenames_subset_8192_v2.csv")
 
    writer = SummaryWriter()
    for epoch in range(1, args['epoch'] + 1):
        train_loss = train(args, model, device, train_set, optimizer,scheduler1, criterion, epoch,)
        test_acc, test_loss = test(args, model, device, test_set,criterion)
        print('Dev accuracy ', test_acc)
        savemodel(model,epoch)
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Loss/test', train_loss, epoch)

        writer.add_scalar('Acc/test', test_acc, epoch)

In [ ]:
if __name__ == '__main__':
    args = {
        'batch_size': 32,
        'log_interval': 30,
        'lr': 0.1,
        'epoch': 300
    }
    main(args)